In [ ]:
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import pandas as pd

COLUMNS = ['product_name', 'product_price', 'product_rating', 'product_review_count', 'product_url']

def set_chrome_options():
    """Sets chrome options for Selenium.
    Chrome options for headless browser is enabled.
    """
    chrome_options = uc.ChromeOptions()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_prefs = {}
    chrome_options.experimental_options["prefs"] = chrome_prefs
    chrome_prefs["profile.default_content_settings"] = {"images": 2}
    return chrome_options

def page_url(page_number):
    url = f'https://www.amazon.com/s?i=computers-intl-ship&bbn=16225007011&rh=n%3A16225007011%2Cn%3A172456&page={page_number}'
    return url

def get_soup(url):
    driver = uc.Chrome(set_chrome_options())
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.quit()
    return soup

def get_product_infos(soup):
    products = []
    all_products_infos = soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small')
    for product in all_products_infos:
        product_name = product.select('span.a-size-base-plus.a-color-base.a-text-normal')[0].text if product.select('span.a-size-base-plus.a-color-base.a-text-normal') else None
        product_price = product.select('span.a-offscreen')[0].text if product.select('span.a-price-whole') else None
        product_rating = product.select('span.a-icon-alt')[0].text.split(' ')[0] if product.select('span.a-icon-alt') else None
        product_review_count = product.select('span.a-size-base')[0].text if product.select('span.a-size-base') else None
        product_url = 'https://www.amazon.com' + product.select('a.a-link-normal.a-text-normal')[0]['href'] if product.select('a.a-link-normal.a-text-normal') else None 
        product_info = [product_name, product_price, product_rating, product_review_count, product_url]
        products.append(product_info)
    return products

df = pd.DataFrame(columns=COLUMNS)
page_number = 1
soup_old = None
soup_new = get_soup(url=page_url(page_number=page_number))
while soup_old != soup_new:
    products = get_product_infos(soup=soup_new)
    df = pd.concat([df, pd.DataFrame(products, columns=COLUMNS)], ignore_index=True)
    soup_old = soup_new
    page_number += 1
    soup_new = get_soup(url=page_url(page_number))
    
df = df.drop_duplicates(subset=COLUMNS).reset_index(drop=True)

In [2]:
df

,product_name,product_price,product_rating,product_review_count,product_url
0,"VEVOR 9U Wall Mount Network Server Cabinet, 15...",$129.99,3.1,5,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1,"Onten USB C Hub,7Port USB 3.0 Hub,USB C to USB...",$19.99,4.5,11,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
2,"SSK 100W USB-C HUB, USB3.2 Gen 2 10Gbps 5 in 1...",$24.99,4.3,27,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
3,HP 67 Black/Tri-color Ink Cartridges (2 Count ...,None,4.7,"39,669",https://www.amazon.com/HP-Cartridges-Black-Tri...
4,SAMSUNG EVO Select Micro SD-Memory-Card + Adap...,$17.99,4.7,"55,949",https://www.amazon.com/SAMSUNG-microSDXC-Expan...
...,...,...,...,...,...
325,"Ozeino Gaming Headset PS4 Headset, Xbox Headse...",$22.98,4.3,"1,066",https://www.amazon.com/Surround-Headphones-Can...
326,"IVANKY VESA Certified DisplayPort Cable, 6.6ft...",$12.99,4.7,"27,459",https://www.amazon.com/IVANKY-Certified-Displa...
327,"BENFEI USB C HUB 7in1, USB C HUB Multiport Ada...",$16.19,4.5,"1,971",https://www.amazon.com/sspa/click?ie=UTF8&spc=...
328,"Mobeetek USB C Hub 10 in 1, with HDMI Adapter ...",$34.99,5.0,1,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
